In [1]:
# %%
# %%
from datasets import load_dataset
import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from utils import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
import logging
import sys
import transformers
import torch
import time
import argparse
from tqdm import tqdm
import string
from RoBERTa import *
from transformers import RobertaTokenizer
from transformers import AutoTokenizer


# %%

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser("main")

parser.add_argument('--train_num_points', type=int,             default = 1000,              help='train data number')
parser.add_argument('--valid_num_points', type=int,             default = 100,              help='validation data number')
parser.add_argument('--report_num_points',type=int,             default = 500,              help='report number')
parser.add_argument('--model_name',       type=str,             default = 'roberta-base',   help='model name')
parser.add_argument('--max_length',       type=int,             default=128,                help='max_length')
parser.add_argument('--batch_size',       type=int,             default=64,                  help='Batch size')
parser.add_argument('--num_workers',      type=int,             default=0,                  help='num_workers')
parser.add_argument('--epochs',           type=int,             default=1,                  help='num of epochs')
parser.add_argument('--lr',               type=float,           default=1e-5,               help='lr')
parser.add_argument('--gamma',            type=float,           default=1,                  help='lr*gamma after each test')

args = parser.parse_args(args=[])#(args=['--batch_size', '8',  '--no_cuda'])#used in ipynb
print(args)

dataset = load_dataset('glue', 'mnli')

print('\n Property of dataset:')
print('train set size: ',len(dataset['train']))
print('validation_matched set size: ',len(dataset['validation_matched']))
print('validation_mismatched set size: ',len(dataset['validation_mismatched']))
print('test_matched set size: ',len(dataset['test_matched']))
print('test_mismatched set size: ',len(dataset['test_mismatched']))
# %%
# %%



# %%
train = dataset['train'][:args.train_num_points]
valid = dataset['validation_matched'][-args.valid_num_points:]

tokenizer = RobertaTokenizer.from_pretrained(args.model_name)
#mnli
#The Multi-Genre Natural Language Inference Corpus is a crowdsourced collection of sentence pairs with textual entailment annotations. Given a premise sentence and a hypothesis sentence, the task is to predict whether the premise entails the hypothesis (entailment), contradicts the hypothesis (contradiction), or neither (neutral). The premise sentences are gathered from ten different sources, including transcribed speech, fiction, and government reports. The authors of the benchmark use the standard test set, for which they obtained private labels from the RTE authors, and evaluate on both the matched (in-domain) and mismatched (cross-domain) section. They also uses and recommend the SNLI corpus as 550k examples of auxiliary training data.

# %%
train_data = get_Dataset(train, tokenizer,max_length=args.max_length)
train_dataloader = DataLoader(train_data, sampler= SequentialSampler(train_data), 
                        batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)
valid_data = get_Dataset(valid, tokenizer,max_length=args.max_length)
valid_dataloader = DataLoader(valid_data, sampler=SequentialSampler(valid_data), 
                        batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)

# %%

model = TextClassifier(args).to(device)
model.train(train_dataloader,valid_dataloader,valid,device)




# %%





Namespace(batch_size=64, epochs=1, gamma=1, lr=1e-05, max_length=128, model_name='roberta-base', num_workers=0, report_num_points=500, train_num_points=1000, valid_num_points=100)

 Property of dataset:
train set size:  392702
validation_matched set size:  9815
validation_mismatched set size:  9832
test_matched set size:  9796
test_mismatched set size:  9847


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForS



hypothesis: John Knox has faded into obscurity with no memorials made for him.
original label: 2
one-word-replaced hypothesis: here Knox has faded into obscurity with no memorials made for him.
premise: Outside the cathedral you will find a statue of John Knox with Bible in hand.
pred label tensor([0], device='cuda:0')
one-word-replaced hypothesis: slow Knox has faded into obscurity with no memorials made for him.
premise: Outside the cathedral you will find a statue of John Knox with Bible in hand.
pred label tensor([0], device='cuda:0')
one-word-replaced hypothesis: was, Knox has faded into obscurity with no memorials made for him.
premise: Outside the cathedral you will find a statue of John Knox with Bible in hand.
pred label tensor([0], device='cuda:0')
one-word-replaced hypothesis: Dolphins Knox has faded into obscurity with no memorials made for him.
premise: Outside the cathedral you will find a statue of John Knox with Bible in hand.
pred label tensor([0], device='cuda:0')
o

In [ ]:
word_label_sensitivity(valid, 10, model, device)



hypothesis: John Knox has faded into obscurity with no memorials made for him.
original label: 2
one-word-replaced hypothesis: here Knox has faded into obscurity with no memorials made for him.
premise: Outside the cathedral you will find a statue of John Knox with Bible in hand.
pred label tensor([2], device='cuda:0')
one-word-replaced hypothesis: slow Knox has faded into obscurity with no memorials made for him.
premise: Outside the cathedral you will find a statue of John Knox with Bible in hand.
pred label tensor([2], device='cuda:0')
one-word-replaced hypothesis: was, Knox has faded into obscurity with no memorials made for him.
premise: Outside the cathedral you will find a statue of John Knox with Bible in hand.
pred label tensor([2], device='cuda:0')
one-word-replaced hypothesis: Dolphins Knox has faded into obscurity with no memorials made for him.
premise: Outside the cathedral you will find a statue of John Knox with Bible in hand.
pred label tensor([2], device='cuda:0')
o

KeyboardInterrupt: 

In [ ]:
import logging    # first of all import the module
from datetime import datetime

logfilename = datetime.now().strftime('./logs/logfile_%H_%M_%d_%m_%Y.log')

logging.basicConfig(filename=logfilename, filemode='w', format='%(name)s - %(levelname)s - %(message)s')
logging.warning("warning")
logging.info("info")
# %%